In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.cm as cm
import matplotlib.pyplot as plt

import matplotlib
import matplotlib.font_manager

In [3]:
#缓存（cache）装饰器
from requests import get
from functools import wraps

def check_cache(f):
    @wraps(f)
    def wrapper(src:str, dst:str, min_size=100) -> Path:
        url = Path(src) 
        fn  = url.name  
        dsn = Path(f"{dst}/{fn}") 
        if dsn.is_file() and dsn.stat().st_size > min_size:
            print(f"+ {dsn} found locally!")
            return(dsn)
        else:
            print(f"+ {dsn} not found, downloading!")
            return(f(src, dsn))
    return wrapper

In [4]:
#下载函数
@check_cache
def cache_data(src:Path, dst:Path) -> str:
    """Downloads a remote file.
    Returns
    -------
    str
        A string representing the local location of the file.
    """
      
    # Create...
    if not dst.parent.exists():
        dst.parent.mkdir(parents=True, exist_ok=True)
        
    # Download and write the file
    with dst.open(mode='wb') as file:
        response = get(src)
        file.write(response.content)
        
    print(' + Done downloading...')

    return dst.resolve()

In [5]:
help(cache_data)

Help on function cache_data in module __main__:

cache_data(src: pathlib.Path, dst: pathlib.Path) -> str
    Downloads a remote file.
    Returns
    -------
    str
        A string representing the local location of the file.



In [6]:
#下载/读取地理数据
ddir  = Path('data/geo')
spath = 'https://github.com/jreades/fsds/blob/master/data/src/' # source path

boros = gpd.read_file( cache_data(spath + 'Boroughs.gpkg?raw=true', ddir.resolve()) )

print('Done.')

+ /home/jovyan/work/The-Universe-s-Premier-F/data/geo/Boroughs.gpkg?raw=true not found, downloading!
 + Done downloading...
Done.


/opt/conda/lib/python3.12/site-packages/pyogrio/raw.py:198: RuntimeWarning: File /home/jovyan/work/The-Universe-s-Premier-F/data/geo/Boroughs.gpkg?raw=true has GPKG application_id, but non conformant file extension
  return ogr_read(


In [15]:
# Input: date + city
ymd  = '20250615'
city = 'London'
host = "https://raw.githubusercontent.com"
repo = "yjzhou217/The-Universe-s-Premier-F/main"

url  = f"{host}/{repo}/{ymd}-{city}-listings.csv.gz"
print(url)

# 创建 data/raw 文件夹
Path("data/raw").mkdir(parents=True, exist_ok=True)

# load csv
file_path = cache_data(url, Path("data/raw").resolve())

df = pd.read_csv(file_path, compression="gzip", low_memory=False)
print(f"Data frame is {df.shape[0]:,} x {df.shape[1]}")


https://raw.githubusercontent.com/yjzhou217/The-Universe-s-Premier-F/main/20250615-London-listings.csv.gz
+ /home/jovyan/work/The-Universe-s-Premier-F/data/raw/20250615-London-listings.csv.gz found locally!
Data frame is 96,651 x 79
